In [13]:
# Create virtual environment
!python3 -m venv venv
!source venv/bin/activate

# Install base packages
!pip install jupyter numpy pandas matplotlib
!pip install sounddevice scipy librosa
!pip install torch torchaudio
!pip install openai elevenlabs
!pip install ipywidgets ipywebrtc
!pip install whisper-openai

# For Jupyter widgets
!jupyter nbextension install --py widgetsnbextension --user
!jupyter nbextension enable widgetsnbextension --user --py


Error: Refusing to create a venv in /Users/soumo141/Documents/sample_projects/AI:ML/voice-scam-shield/venv because it contains the PATH separator :.
zsh:source:1: no such file or directory: venv/bin/activate
  Using cached jupyter-1.1.1-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached numpy-2.3.2-cp313-cp313-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached pandas-2.3.1-cp313-cp313-macosx_11_0_arm64.whl.metadata (91 kB)
  Using cached matplotlib-3.10.5-cp313-cp313-macosx_11_0_arm64.whl.metadata (11 kB)
  Using cached notebook-7.4.5-py3-none-any.whl.metadata (10 kB)
  Using cached jupyter_console-6.6.3-py3-none-any.whl.metadata (5.8 kB)
  Using cached nbconvert-7.16.6-py3-none-any.whl.metadata (8.5 kB)
  Using cached ipykernel-6.30.1-py3-none-any.whl.metadata (6.2 kB)
  Using cached jupyterlab-4.4.5-py3-none-any.whl.metadata (16 kB)
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl.metadata (8.4 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)


In [2]:
# main.py
from fastapi import FastAPI, WebSocket, WebSocketDisconnect
from fastapi.middleware.cors import CORSMiddleware
import uvicorn
import json

from services import (
    TranscriptionService,
    AntiSpoofingService,
    ScamDetectionService,
    AlertService
)

app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # for development
    allow_methods=["*"],
    allow_headers=["*"],
)

# Instantiate services once
transcriber = TranscriptionService()
antispoofer = AntiSpoofingService()
scam_detector = ScamDetectionService()
alerter = AlertService()

@app.websocket("/ws/stream")
async def audio_stream(ws: WebSocket):
    await ws.accept()
    try:
        while True:
            # Receive audio chunk from client
            msg = await ws.receive_json()
            # msg = { "session_id": "...", "audio": [float,...], "timestamp": 12345 }
            session_id = msg["session_id"]
            audio_bytes = bytes(msg["audio"])

            # 1) Transcribe
            transcript_res = await transcriber.transcribe_audio(audio_bytes)
            text = transcript_res["text"]
            lang = transcript_res["language"]

            # 2) Anti-Spoofing
            spoof_res = await antispoofer.detect_synthetic_voice(audio_bytes)

            # 3) Scam Detection
            scam_res = await scam_detector.analyze_transcript(text, lang)

            # 4) Combine risk
            final_risk = (
                "SCAM"
                if spoof_res["is_synthetic"] or scam_res["risk_level"] == "SCAM"
                else scam_res["risk_level"]
            )

            # 5) Generate TTS alert if needed
            audio_alert = None
            if final_risk in ("SUSPICIOUS", "SCAM"):
                msg_text = alerter.get_alert_message(final_risk, lang, scam_res["reasons"])
                audio_alert = await alerter.generate_audio_alert(msg_text, lang)

            # 6) Build response
            response = {
                "session_id": session_id,
                "transcript": transcript_res,
                "antispoofing": spoof_res,
                "scam_analysis": scam_res,
                "final_risk": final_risk,
                "audio_alert": audio_alert.hex() if audio_alert else None
            }
            await ws.send_text(json.dumps(response))

    except WebSocketDisconnect:
        print(f"Session {session_id} disconnected")

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)


ModuleNotFoundError: No module named 'fastapi'

In [3]:
# VoiceScamShield.tsx
'use client';
import { useEffect, useRef, useState } from 'react';
import { MicVAD } from '@ricky0123/vad-web';

interface ServerResponse {
  session_id: string;
  transcript: { text: string; language: string; };
  antispoofing: { is_synthetic: boolean; confidence: number; };
  scam_analysis: { risk_level: string; reasons: string[]; };
  final_risk: string;
  audio_alert: string | null; // hex string
}

export default function VoiceScamShield() {
  const wsRef = useRef<WebSocket>();
  const vadRef = useRef<any>();
  const [sessionId] = useState(() => crypto.randomUUID());
  const [risk, setRisk] = useState('SAFE');
  const [transcript, setTranscript] = useState('');
  
  useEffect(() => {
    // Initialize WebSocket
    wsRef.current = new WebSocket('ws://localhost:8000/ws/stream');
    wsRef.current.onmessage = (evt) => {
      const res: ServerResponse = JSON.parse(evt.data);
      setTranscript(prev => prev + ' ' + res.transcript.text);
      setRisk(res.final_risk);
      if (res.audio_alert) {
        // Play alert audio
        const audioBytes = new Uint8Array(res.audio_alert.match(/../g)!.map(h => parseInt(h, 16)));
        const blob = new Blob([audioBytes], { type: 'audio/wav' });
        const url = URL.createObjectURL(blob);
        new Audio(url).play();
      }
    };

    // Initialize VAD
    MicVAD.new({
      onSpeechEnd: (audio: Float32Array) => {
        // Send chunk to server
        if (wsRef.current?.readyState === WebSocket.OPEN) {
          wsRef.current.send(JSON.stringify({
            session_id: sessionId,
            audio: Array.from(audio),
            timestamp: Date.now()
          }));
        }
      },
    }).then(vad => { vadRef.current = vad; vad.start(); });

    return () => {
      wsRef.current?.close();
      vadRef.current?.destroy();
    };
  }, [sessionId]);

  return (
    <div>
      <h2>Live Risk: <span>{risk}</span></h2>
      <p>Transcript: {transcript}</p>
    </div>
  );
}


SyntaxError: invalid syntax (94680872.py, line 3)